# Лабораторная работа №5. Метрики качества классификации

Выполнили: Рутин Василий, Гордеев Станислав

Все лабораторные работы доступны по адресу: https://github.com/Omenstudio/ML_IFMO_LABS

## 1. Постановка задачи

1. Прочитать теоретическую часть
2. Для двух любых методов классификации из предыдущих работ и своего набора данных посчитать следующие метрики качества:<br>
    a. Точность классификации (Classification Accuracy)<br>
    b. Логарифм функции правдоподобия (Logarithmic Loss)<br>
    c. Область под кривой ошибок (Area Under ROC Curve)<br>
    d. Матрица неточностей (Confusion Matrix)<br>
    e. Отчет классификации (Classification Report)<br>
3. Для более точных результатов использовать кросс-валидацию
4. Сравнить применимость используемых классификаторов, основываясь на полученных метриках

## 2. Исходные данные

Задача классификации: определение наличия заболевания (диабет) у женщин.<br>
Датасет доступен по адресу: https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

Количество записей: 768<br>
Количество атрибутов: 8 плюс класс.<br>
Все атрибуты представлены числовыми значениями:
1. Количество беременностей (шт.)
2. Концентрация плазмы глюкозы (ммоль/л)
3. Диастолическое давление (мм. рт. столба)
4. Толщина кожной складки (мм.)
5. Содержание инсулина (мкЕд/мл)
6. Индекс массы тела (кг/м2)
7. Предрасположенность по родословной линии (Diabetes pedigree function)
8. Возраст (лет)
9. Класс: наличие диабета (0, 1)

## 3. Ход работы

Функцию, считывающую датасет, возьмем из предыдущих лабораторных работ.

In [1]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


def load_dataset(split_ratio=None, normalize=True):
    dataset_full = np.loadtxt(open("../pima-indians-diabetes.data.csv", "r"), delimiter=",", skiprows=0, dtype=np.float64)
    inputs = dataset_full[:, :-1]
    outputs = dataset_full[:, -1]
    outputs = outputs.astype(np.int64, copy=False)
    if split_ratio is None:
        return inputs, outputs, None, None
    inputs_train, inputs_test, outputs_train, outputs_test \
        = train_test_split(inputs, outputs, test_size=split_ratio, random_state=random.randint(0, 1000))
    if normalize:
        std_scale = preprocessing.StandardScaler().fit(inputs_train)
        inputs_train = std_scale.transform(inputs_train)
        inputs_test = std_scale.transform(inputs_test)
    return inputs_train, outputs_train, inputs_test, outputs_test

Далее напишем метод, который будет принимать на вход модель, входные и выходные данные, и выводить в консоль все метрики качества. Для повышения точности будем использовать кросс-валидацию.

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


def print_score(model, inputs, outputs):
    print('Model: {}'.format(model))
    print('CrossValidation results:')
    #
    scorings = ['accuracy', 'neg_log_loss', 'roc_auc']
    for scoring in scorings:
        # кросс-валидация здесь!
        results = cross_val_score(model, inputs, outputs, cv=KFold(n_splits=10), scoring=scoring)
        print('{}: {:.3f} ({:.3f})'.format(scoring, results.mean(), results.std()))
    #
    X_train, X_test, Y_train, Y_test = train_test_split(inputs, outputs, test_size=0.4)
    model.fit(X_train, Y_train)
    predicted = model.predict(X_test)
    #
    print('ConfusionMatrix:\n', confusion_matrix(Y_test, predicted))
    print('ClassificationReport:\n', classification_report(Y_test, predicted))
    #
    print('--------------')

Тестируем Наивный Байесовский алгоритм и метод К ближайших соседей.

In [3]:
def main():
    inputs, outputs, _, _ = load_dataset()
    print_score(GaussianNB(), inputs, outputs)
    print_score(KNeighborsClassifier(n_neighbors=75), inputs, outputs)


main()

Model: GaussianNB(priors=None)
CrossValidation results:
accuracy: 0.755 (0.043)
neg_log_loss: -0.619 (0.171)
roc_auc: 0.818 (0.045)
ConfusionMatrix:
 [[161  51]
 [ 31  65]]
ClassificationReport:
              precision    recall  f1-score   support

          0       0.84      0.76      0.80       212
          1       0.56      0.68      0.61        96

avg / total       0.75      0.73      0.74       308

--------------
Model: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=75, p=2,
           weights='uniform')
CrossValidation results:
accuracy: 0.746 (0.051)
neg_log_loss: -0.531 (0.058)
roc_auc: 0.786 (0.068)
ConfusionMatrix:
 [[190  19]
 [ 76  23]]
ClassificationReport:
              precision    recall  f1-score   support

          0       0.71      0.91      0.80       209
          1       0.55      0.23      0.33        99

avg / total       0.66      0.69      0.65       308

--------------


## 4. Заключение

В ходе лабораторной работы посчитаны основные метрики качества для Наивного Байесовского алгоритма и метода К средних. Сделаны следующие выводы:<br>
1. Точность GaussianNB(0.755) выше, чем KNN(0.746). Правда, всего на 0.9%. Следуеть посмотреть на другие метрики, возможно, они помогут выбрать лучший алгоритм. 
2. Результаты log_loss и roc_auc говорят в пользу KNN.
3. Анализ матрицы неточностей показывает, что неверных срабатываний в обоих алгоритмах примерно поровну, а вот положительно угаданных больше в KNN. 
4. Метрика точности (precision) говорит, что "доверять" лучше GaussianNB.
5. Метрика полноты (recall) говорит, что KNN хорошо "угадывает" отсутствие диабета и очень плохо его наличие.